<a href="https://colab.research.google.com/github/mahsabargahi/google-colab/blob/main/LR_HW_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn # in case you need to upgrade sklearn

     |████████████████████████████████| 23.1 MB 5.4 kB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
! pip uninstall statsmodels
! pip install statsmodels

Found existing installation: statsmodels 0.10.2
Uninstalling statsmodels-0.10.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/statsmodels-0.10.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/statsmodels/*
Proceed (y/n)? y
  Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 9.5 MB 13.2 MB/s 


In [ ]:
## mount the Google drive to get access to files in your drive ##
from google.colab import drive
drive.mount('/content/drive')

## set the folder that stores your files and data as your current directory ##
%cd /content/drive/My\ Drive/Colab\ Notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [ ]:
## import libraries and tools ##

import numpy as np # linear algebra
import pandas as pd # data processing and CSV file I/O 

import seaborn as sns

from matplotlib import rcParams
import matplotlib.pyplot as plt

import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA

import missingno  # missing data visualization

import scipy as sp
from scipy import stats

import statsmodels.api as sm
import statsmodels.stats.stattools as smt

In [ ]:
## read the dataset in csv format ##
df = pd.read_csv('Data/Concrete_Data.csv')
df.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [ ]:
## change the name of column "csMpa" to "strength" and return a new dataset
df.rename(columns={'csMPa': 'strength'}, inplace=True)
print(df.shape)
df.head()

(1030, 9)


,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [ ]:
# separate the feature data X from the response data y
X = df.iloc[:,:-1] # all columns except for the last column
y = df.iloc[:,-1] # the last column is the response variable

In [ ]:
## split the dataset into training and testing datasets using a 80%-20% randomized split ##
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.2, random_state=15)

In [ ]:
# dimension of the training dataset
X_train.shape 

(824, 8)

In [ ]:
# assembly model evaluation metrics together
def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    #return mae, mse, rmse, r2_square
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R Square:', r2_square)
    print('__________________________________')

Linear Regression Model

In [ ]:
# call the linear regressor
lr = LinearRegression()

In [ ]:
# evaluate the goodness-of-fit of the regression model in training using cross-validation
r2_train = cross_val_score(lr, X_train, y_train, cv =5, scoring = 'r2').mean()
print(f'the average r-square value is {r2_train*100 :.2f}%')

the average r-square value is 60.51%


In [ ]:
# fit the linear regression model
lr_model = lr.fit(X_train,y_train)

In [ ]:
# intercept, b_0
print(f'The intercept is {lr_model.intercept_:.3f}')

The intercept is -20.810


In [ ]:
# coefficients, b1,..., b8
coeff_df = pd.DataFrame(lr_model.coef_, ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8'], columns=['Coefficient'])
coeff_df.round(4)

,Coefficient
PC1,0.1205
PC2,0.1049
PC3,0.0869
PC4,-0.1637
PC5,0.2563
PC6,0.0175
PC7,0.0206
PC8,0.1224


In [ ]:
## You can also find coefficient using linear algebra
beta_hat = np.linalg.inv(Z_train.T @ Z_train) @ Z_train.T @ y_train
print(f'Regression coefficients are:',beta_hat)

Regression coefficients are: [-2.08098188e+01  1.20523374e-01  1.04905858e-01  8.68975319e-02
 -1.63716537e-01  2.56274595e-01  1.75170441e-02  2.05659761e-02
  1.22422892e-01]


In [ ]:
# calculate standard errors of efficients using linear algebra
n = X_train.shape[0]
p = X_train.shape[1]
y_train_pred = lr_model.predict(X_train)
residuals = y_train - y_train_pred
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (n-p-1)

var_beta_hat =np.linalg.inv(Z_train.T @ Z_train)*sigma_squared_hat
for p in range(p+1):
  standard_error = var_beta_hat[p, p] ** 0.5
  print(f"SE(beta_hat[{p}]): {standard_error}")

SE(beta_hat[0]): 29.90573169463212
SE(beta_hat[1]): 0.009463569039049588
SE(beta_hat[2]): 0.011154546395421771
SE(beta_hat[3]): 0.013891100520166816
SE(beta_hat[4]): 0.04559850632907878
SE(beta_hat[5]): 0.1063315713755482
SE(beta_hat[6]): 0.010462328343609127
SE(beta_hat[7]): 0.012034016809344452
SE(beta_hat[8]): 0.006224309187148917


In [ ]:
# margin of error for coefficients

alpha = 0.05 # level of significance. You can change to your own value
t_val = sp.stats.t.ppf(1-alpha/2, n-p-1)
for p in range(p+1):
  me=var_beta_hat[p, p] ** 0.5*t_val
  print(f'margin of error[beta_hat[{p}]:{me}')

margin of error[beta_hat[0]:58.70133274760304
margin of error[beta_hat[1]:0.018575840939577367
margin of error[beta_hat[2]:0.021895024883265415
margin of error[beta_hat[3]:0.027266549509339624
margin of error[beta_hat[4]:0.08950435054218671
margin of error[beta_hat[5]:0.20871600857744335
margin of error[beta_hat[6]:0.02053628460537263
margin of error[beta_hat[7]:0.023621318890598264
margin of error[beta_hat[8]:0.012217565798076547


In [ ]:
# evaluate the model on the teting dataset
print_evaluate(y_test, lr_model.predict(X_test))

MAE: 8.274041729659364
MSE: 114.16396346419538
RMSE: 10.684753785848104
R Square: 0.5570208623821267
__________________________________
